In [100]:
import pandas as pd
df1 = pd.read_csv("/scratch/arjunth2001/train_df.csv")#.head(500)
df2 = pd.read_csv("/scratch/arjunth2001/dev_df.csv")#.head(200)

In [101]:
dataset1= {
    'question_ids': [],
    'questions': [],
    'contexts' : [],
    'context_ids': []
}
dataset2 = {
    'question_ids': [],
    'questions': [],
    'contexts': [],
    'context_ids': []
}
for i in range(len(df1)):
    dataset1['question_ids'].append(i)
    dataset1['questions'].append(df1['q'][i])
    dataset1['contexts'].append(df1['para'][i])
    dataset1['context_ids'].append(i)
for i in range(len(df2)):
    dataset2['question_ids'].append(i)
    dataset2['questions'].append(df2['q'][i])
    dataset2['contexts'].append(df2['para'][i])
    dataset2['context_ids'].append(i)

In [102]:
from tqdm.auto import tqdm
from dotdict import DotDictify
from dictionary import Dictionary
from tokenizer import SpacyTokenizer
from multiprocessing.pool import ThreadPool as Pool
import os
import json
import itertools

TOK = None
# Each process has its own tokenizer


def init_tokenizer(annotators):
    global TOK
    TOK = SpacyTokenizer(annotators=annotators)

# Multiprocessing requires global function


def tokenize(text):
    global TOK
    return TOK.tokenize(text)


# def tokenize_all(texts, annotators, num_workers=None):
#     """Tokenization might take a long time, even when done in parallel"""
#     init_tokenizer(annotators)
#     tokens = map(tokenize, texts)
#     return list(tokens)
def tokenize_all(texts, annotators, num_workers=None):
    """Tokenization might take a long time, even when done in parallel"""
    workers = Pool(num_workers, init_tokenizer,
                   initargs=[annotators])
    tokens = workers.map(tokenize, texts)
    workers.close()
    workers.join()
    return tokens


In [103]:
def get_output(dataset,contexts,questions):
    examples = []
    for qid, cid in tqdm(enumerate(dataset['context_ids'])):
        examples.append({
            'id': dataset['question_ids'][qid],
            'question': {key: questions[qid][key] for key in ['tokens', 'lemma']},
            'context_id': cid,
            'answers': {'spans': [], 'texts': []}
        })
    output = {'contexts': contexts, 'examples': examples}
    return output


In [104]:
if __name__=="__main__":
    questions1 = tokenize_all(dataset1['questions'], [
        'lemma'],12)
    contexts1 = tokenize_all(dataset1['contexts'], [
        'lemma', 'pos', 'ner'],12)
    questions2 = tokenize_all(dataset2['questions'], [
        'lemma'], 12)
    contexts2 = tokenize_all(dataset2['contexts'], [
        'lemma', 'pos', 'ner'], 12)
    


In [105]:
output1 = get_output(dataset1,contexts1,questions1)
output2 = get_output(dataset2,contexts2,questions2)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [106]:
import json
import logging
import os

import torch
import torch.nn.functional as F

from tqdm import tqdm
import drqa
import utils
from dictionary import Dictionary
from dataset import ReadingDataset, BatchSampler
from dotdict import DotDictify
args = {
    "seed": 42,
    "data": "/scratch/arjunth2001/data",
    "max_tokens": 16000,
    "batch_size": 32,
    "num_workers": 4,
    "max_epoch": 400,
    "clip_norm": 10,
    "lr": 2e-6,
    "momentum": 0.99,
    "weight_decay": 0.0,
    "lr_shrink": 0.1,
    "min_lr": 1e-8,
    "log_file": "/scratch/arjunth2001/logs/train.log",
    "tune_embed": 1000,
    "checkpoint_dir": "./models",
    'embed_dim': 300,
    'embed_path': '/scratch/arjunth2001/data/glove.840B.300d.txt',
    'hidden_size': 128,
    'context_layers': 3,
    'question_layers': 3,
    'dropout': 0.4,
    'bidirectional': True,
    'concat_layers': True,
    'question_embed': True,
    'use_in_question': True,
    'use_lemma': True,
    'use_pos': True,
    'use_ner': True,
    'use_tf': True,

}
args = DotDictify(args)

In [107]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print('device: ' + str(device))

torch.manual_seed(args.seed)

# Load a dictionary
dictionary = Dictionary.load(os.path.join(args.data, 'dict.txt'))
print(
    'Loaded a word dictionary with {} words'.format(len(dictionary)))

# Load a training and validation dataset
with open(os.path.join(args.data, 'train.json')) as file:
    train_contents = json.load(file)
    train_dataset = ReadingDataset(
        args, train_contents['contexts'], train_contents['examples'], dictionary, skip_no_answer=True, single_answer=True)

with open(os.path.join(args.data, 'dev.json')) as file:
    contents = json.load(file)
    valid_dataset = ReadingDataset(
        args, contents['contexts'], contents['examples'], dictionary, feature_dict=train_dataset.feature_dict, skip_no_answer=True, single_answer=True
    )
model = drqa.DrQA.build_model(args, dictionary).to(device)
print('Built a model with {} parameters'.format(
    sum(p.numel() for p in model.parameters())))
# Build an optimizer and a learning rate schedule
optimizer = torch.optim.Adamax(
    model.parameters(), args.lr, weight_decay=args.weight_decay)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='max', patience=3, factor=args.lr_shrink)

# Load last checkpoint if one exists
_ = utils.load_checkpoint(args, model, optimizer, lr_scheduler, device)


device: cuda
Loaded a word dictionary with 29934 words
Loading Embedding..
Loaded 29547 / 29934 word embeddings (98.71%)
Built a model with 13096101 parameters
Loaded Checkpoint


In [108]:
contents1 = output1
test_dataset1 = ReadingDataset(
    args, contents1['contexts'], contents1['examples'], dictionary, feature_dict=train_dataset.feature_dict, skip_no_answer=False, single_answer=True
)


In [109]:
contents2 = output2
test_dataset2 = ReadingDataset(
    args, contents2['contexts'], contents2['examples'], dictionary, feature_dict=train_dataset.feature_dict, skip_no_answer=False, single_answer=True
)


In [110]:
def run(model,test_dataset,df):
    scores_ = []
    answers_ = []
    indxs =[]
    ids = []
    model.eval()
    test_loader = torch.utils.data.DataLoader(
        test_dataset, num_workers=args.num_workers, collate_fn=test_dataset.collater, shuffle=False,
        batch_sampler=BatchSampler(
            test_dataset, args.max_tokens, args.batch_size, shuffle=False, seed=args.seed)
    )
    progress_bar = tqdm(
        test_loader, desc='Testing', leave=False)

    for batch_id, sample in enumerate(progress_bar):
        sample = utils.move_to_device(sample, device)
        with torch.no_grad():
            start_scores, end_scores = model(
                sample['context_tokens'], sample['question_tokens'],
                context_features=sample['context_features']
            )

            start_pred, end_pred, scores = model.decode(
                start_scores, end_scores, max_len=15)

            for i, (start_ex, end_ex, score) in enumerate(zip(start_pred, end_pred, scores)):
                context = test_dataset.contexts[test_dataset.context_ids[sample['id'][i]]]
                start_idx = context['offsets'][start_ex][0]
                end_idx = context['offsets'][end_ex][1]
                text_pred = context['text'][start_idx: end_idx]
                ids.append(sample['id'][i])
                answers_.append(text_pred)
                scores_.append(score.item())
                indxs.append((start_idx,end_idx))
    temp = pd.DataFrame()
    temp['answer'] = answers_
    temp['score'] = scores_
    temp['idxs'] = indxs
    temp["ids"] = ids
    temp.sort_values(by=['ids'],inplace=True)
    temp.reset_index(drop=True,inplace=True)
    df["answer"] = temp["answer"]
    df["score"] = temp["score"]
    df["idxs"] = temp["idxs"]
    df["ids"] = temp["ids"]
    return df

In [111]:
df1 = run(model,test_dataset1,df1)

Testing:   0%|          | 0/16 [00:00<?, ?it/s]/home2/arjunth2001/Wikipedia-Open-QnA/module_2/drqa.py:114: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  pred_start.append(max_idx // scores.size(0))


In [112]:
df1.to_csv("/scratch/arjunth2001/output1.csv",index=False)

In [113]:
df2 = run(model, test_dataset2, df2)


In [114]:
df2.to_csv("/scratch/arjunth2001/output2.csv", index=False)
